<a href="https://colab.research.google.com/github/dukes101/LOM-Analytics/blob/main/TeamPerformanceDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read in Data

In [23]:
## Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

## Libraries
#!pip install dash
#!pip install plotly
#!pip install dash-ag-grid
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_ag_grid as dag
import plotly.graph_objects as go

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
## Year by Year Performance Summary for each team
dfYearbyYear = pd.read_csv('/content/drive/My Drive/dfYearbyYear.csv')
metric_list = dfYearbyYear.columns.to_list()[2:]

## Year by Year Averages for entire league
dfYearbyYearAvg = pd.read_csv('/content/drive/My Drive/dfYearbyYearAvg.csv')

## Easiest/Toughest Opponent for each team
dfEasyToughOpp = pd.read_csv('/content/drive/My Drive/dfVersus.csv')

## List all performance metrics for each team
dfAllMetrics = pd.read_csv('/content/drive/My Drive/all_metrics_df.csv')
my_list = list(range(len(dfAllMetrics[dfAllMetrics['Team'] == 'Luca Hurst'])))

## List the top 3 best performance metrics for each team
dfTopMetrics = pd.read_csv('/content/drive/My Drive/top_metrics_df.csv')

## Lists the top 3 worst performance metrics for each team
dfWorstMetrics = pd.read_csv('/content/drive/My Drive/worst_metrics_df.csv')

## Performance Summary for each team (Championships, Top 3 Finishes, Playoff Appearances)
dfTeamPerformance = pd.read_csv('/content/drive/My Drive/historicteamperformance.csv')

# Team Performance Dashboard

In [31]:
app = Dash()
app.layout = [

    html.Div([
    ## Dashboard Title
    html.H1('Player Performance', style={'color': 'white', 'textAlign': 'center'})
    ## Team Dropdown
    ,html.Label('Select Team:', style={'font-size': '20px', 'font-weight': 'bold', 'margin-right': '10px', 'color': 'white'})
    ,dcc.Dropdown(
        options=[{'label': i, 'value': i} for i in dfTeamPerformance['Team'].unique()]
        ,value='Luca Hurst'
        ,id='team-dropdown'
        #,style={'width': '70%'}
        )
    ,html.Div([
    ## Historic Performance Table
    dag.AgGrid(
        id='historic-performance-grid'
        ,rowData=dfTeamPerformance.to_dict('records')
        ,columnDefs=[{'field': 'Metric'}, {'field': 'Value'}]
        ,style={'width': '50%'}
        ,columnSize="responsiveSizeToFit"
        #,dashGridOptions={"headerHeight":0, 'fillHorizontal': True}
    )
    ## Best Performance Metrics Table
    ,dag.AgGrid(
        id='best-performance-grid'
        ,rowData=dfAllMetrics.to_dict('records')
        ,columnDefs=[{'field': 'Metric'}, {'field': 'Value'}, {'field': 'League Rank'}]
        ,style={'width': '50%'}
        ,columnSize="responsiveSizeToFit"
        #,dashGridOptions={"headerHeight":0, 'fillHorizontal': True}
        )
    ], style={'display': 'flex', 'flex-direction': 'row'})
    ## Metric Dropdown
    ,html.Label('Select Metric:', style={'font-size': '20px', 'font-weight': 'bold', 'margin-right': '10px', 'color': 'white'})
    ,dcc.Dropdown(
        options=[{'label': col, 'value': col} for col in metric_list]
        ,value='Points'
        ,id='metric-dropdown'
        #,style={'width': '70%'}
        )
    ## Dynamic Metric Figure
    ,dcc.Graph(
        figure={}
        ,id='metric-by-year-graph'
        #,style={'width': '50%'}
    )])
    ]


## Historic Performance Callback
@callback(
    Output('historic-performance-grid', 'rowData')
    ,Input('team-dropdown', 'value')
)
## Updates historic table
def update_historic_performance_grid(team):
  df_filtered = dfTeamPerformance[dfTeamPerformance['Team'] == team]
  data = [{'Metric': col, 'Value': df_filtered[col].iloc[0]} for col in ['Championships', 'Top 3 Finishes', 'Playoff Appearances']]
  return data

## Best Performance Callback
@callback(
    Output('best-performance-grid', 'rowData')
    ,Input('team-dropdown', 'value')
)
## Updates top metric table
def update_top_metric_table(team):
  df_filtered = dfAllMetrics[dfAllMetrics['Team'] == team]
  data = [{'Metric': df_filtered['Metric'].iloc[row], 'Value': df_filtered['Value'].iloc[row], 'League Rank': df_filtered['League Rank'].iloc[row]} for row in my_list]
  return data

## Metric Figure Callback
@callback(
    Output('metric-by-year-graph', 'figure')
    ,Input('team-dropdown', 'value')
    ,Input('metric-dropdown', 'value')
)

## Updates figure based on team and metric selected
def update_metric_by_year_graph(team, metric):

  # Filter on team
  dfYearbyYearTeam = dfYearbyYear[dfYearbyYear['Team'] == team]

  # Create main line graph for selected metric
  fig1 = px.line(dfYearbyYearTeam, x='Year', y=metric, title=f'{metric} by Year for {team}')
  fig1.update_xaxes(type='category')

  # Add a line for the historic averages
  #avg_df = dfYearbyYearAvg[dfYearbyYearAvg['Year'].isin(dfYearbyYearTeam['Year'])]
  #fig2 = px.line(avg_df, x='Year', y=metric, line_dash='dash', line_color='red')

  #fig = go.Figure(data=fig1.data + fig2.data)
  #fig.update_layout(title=f'{metric} by Year for {team}')
  return fig1

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>